In [ ]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.1-linux-x86_64.tar.gz
!tar -xzf elasticsearch-7.10.1-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.10.1

--2022-03-14 15:28:39--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.10.1-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 318801277 (304M) [application/x-gzip]
Saving to: ‘elasticsearch-7.10.1-linux-x86_64.tar.gz’

elasticsearch-7.10. 100%[===================>] 304.03M  38.5MB/s    in 14s     

2022-03-14 15:28:53 (22.3 MB/s) - ‘elasticsearch-7.10.1-linux-x86_64.tar.gz’ saved [318801277/318801277]



In [ ]:
import os
from subprocess import Popen, PIPE, STDOUT

In [ ]:
es_server = Popen(
    ["elasticsearch-7.10.1/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)

In [ ]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "20c69e357e00",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "lBuU8d-ITDy2bYCeIRJAWA",
  "version" : {
    "number" : "7.10.1",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "1c34507e66d7db1211f66f3513706fdf548736aa",
    "build_date" : "2020-12-05T01:00:33.671820Z",
    "build_snapshot" : false,
    "lucene_version" : "8.7.0",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
import requests
res = requests.get('http://localhost:9200/_cluster/health')

res.json()

{'active_primary_shards': 0,
 'active_shards': 0,
 'active_shards_percent_as_number': 100.0,
 'cluster_name': 'elasticsearch',
 'delayed_unassigned_shards': 0,
 'initializing_shards': 0,
 'number_of_data_nodes': 1,
 'number_of_in_flight_fetch': 0,
 'number_of_nodes': 1,
 'number_of_pending_tasks': 0,
 'relocating_shards': 0,
 'status': 'green',
 'task_max_waiting_in_queue_millis': 0,
 'timed_out': False,
 'unassigned_shards': 0}

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 2.3 MB/s 
     |████████████████████████████████| 3.8 MB 15.0 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 48.8 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 6.5 MB 45.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=fd15c03a031a5e2878525f3d06cea84496ed5a3b7a03929ec35c10b107b4272b
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install elasticsearch==7.10.1

     |████████████████████████████████| 322 kB 3.2 MB/s 


In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
from elasticsearch import Elasticsearch, helpers
import csv
import time
from tqdm import tqdm_notebook

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
es = Elasticsearch(["localhost:9200/"])

In [ ]:
#print new index list
create_response = es.cat.indices()
print(create_response)

In [ ]:
import torch

In [ ]:
url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
dataset_path = "quora_duplicate_questions.tsv"
max_corpus_size = 1000

#Download dataset if needed
if not os.path.exists(dataset_path):
    print("Download dataset")
    util.http_get(url, dataset_path)

#Get all unique sentences from the file
all_questions = {}
with open(dataset_path, encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        all_questions[row['qid1']] = row['question1']
        if len(all_questions) >= max_corpus_size:
            break

        all_questions[row['qid2']] = row['question2']
        if len(all_questions) >= max_corpus_size:
            break

qids = list(all_questions.keys())
questions = [all_questions[qid] for qid in qids]

Download dataset


  0%|          | 0.00/58.2M [00:00<?, ?B/s]

In [ ]:
sentence_embeddings = model.encode(questions)

In [ ]:
for question, embedding in zip(questions[:4], sentence_embeddings[:4]):
    print('Sentence:', question)
    print('Embedding:', list(embedding[:5]) + ['...'])
    print('Embedding\'s length:', len(embedding))
    print('')

Sentence: What is the step by step guide to invest in share market in india?
Embedding: [-0.009722767, -0.3216227, 0.92113924, 0.12629469, 1.0372838, '...']
Embedding's length: 768

Sentence: What is the step by step guide to invest in share market?
Embedding: [0.15146255, -0.20154285, 0.95811677, 0.015940098, 0.8451391, '...']
Embedding's length: 768

Sentence: What is the story of Kohinoor (Koh-i-Noor) Diamond?
Embedding: [0.27386698, 0.4727915, -0.6623533, 0.104528755, 0.5533443, '...']
Embedding's length: 768

Sentence: What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?
Embedding: [0.19313535, 0.091349825, -1.0451186, 0.5032027, 0.6360789, '...']
Embedding's length: 768



In [ ]:
settings = {
  "mappings": {
    "properties": {
      "bert_vector": {
         "type": "dense_vector",
        "dims": 768
      }
    }
  }
}
ques_index = "bert-question"
es.indices.delete(index=ques_index, ignore=[400, 404])
es.indices.create(ques_index, body=settings)

{'acknowledged': True, 'index': 'bert-question', 'shards_acknowledged': True}

In [ ]:
from tqdm import tqdm_notebook

model.eval()

for i, question in tqdm_notebook(enumerate(questions)):
  with torch.no_grad():
    if torch.cuda.is_available():
      torch.cuda.ipc_collect()
      torch.cuda.empty_cache()
    es.index(
      index=ques_index, 
      id=i, 
      body={
          'bert_vector': model.encode(question),
          'text': question
          }
    )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


0it [00:00, ?it/s]

In [ ]:
test_str = 'When will the Pokémon series end?'

#test query knn-search
res = es.search(
    index="bert-question", 
    body={
        "query" : {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, doc['bert_vector']) + 1.0",
                    "params": {"query_vector": list(model.encode(test_str).astype(float))}
                                                   }}}})


print("Got %d Hits:\n\n" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
  print(f"Cosine similarity score: {hit['_score']}  \nid: {hit['_id']}\ntext: {hit['_source']['text']}\n\n")

Got 1000 Hits:


Cosine similarity score: 2.0  
id: 962
text: When will the Pokémon series end?


Cosine similarity score: 1.690942  
id: 518
text: What will happen if I cancel TQWL tickets before the charting is done?


Cosine similarity score: 1.6317922  
id: 519
text: Can we cancel a 'Tatkal waiting list' ticket?


Cosine similarity score: 1.6269788  
id: 279
text: What's life after retirement?


Cosine similarity score: 1.583207  
id: 554
text: How do people die?


Cosine similarity score: 1.5818186  
id: 677
text: How did you quit/stop smoking?


Cosine similarity score: 1.5768082  
id: 456
text: How TQWL is different from CKWL (Tatkal waiting list)?


Cosine similarity score: 1.5729086  
id: 17
text: When do you use "&" instead of "and"?


Cosine similarity score: 1.5725851  
id: 872
text: Why do we fall asleep?


Cosine similarity score: 1.5720441  
id: 760
text: What should I do when my bestfriend is ignoring me?




/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:193: ElasticsearchDeprecationWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
